In [1]:
import os
import sys
import json
import numpy as np
import pandas as pd
import multiprocessing

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import ray
from ray import tune
from ray.tune import CLIReporter
from ray.rllib.agents.dqn import DQNTrainer, DEFAULT_CONFIG

root_dir = os.getcwd().replace('notebooks', '')
# sys.path.append(root_dir)
import numpy as np
import pandas as pd
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', '{:.4f}'.format)
np.set_printoptions(suppress=True, precision=4)
os.environ['PYTHONHASHSEED'] = '0'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"   

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
for gpu_instance in physical_devices: 
    tf.config.experimental.set_memory_growth(gpu_instance, True)

In [3]:
n_cpus = multiprocessing.cpu_count()
n_cpus

24

In [4]:
n_gpus = len(tf.config.list_physical_devices('GPU'))
n_gpus

3

In [5]:
from env_crypto import CryptoEnv

C:\Users\yosou\Documents\BooksWithCode\Binance\Deep Learning\Reinforcement-Learning\RL-ray\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [6]:
pairs = 'ADAUSDT'
lag = 150

In [7]:
url = f'C:/Users/yosou/Documents/BooksWithCode/Binance/Binance-Ai-Bot-Starter/Data/SPOT/{pairs}/{pairs}-5m-data.csv'

In [8]:
data = pd.read_csv(url)
data

,timestamp,open,high,low,close,volume
0,2021-01-01 00:00:00,0.1813,0.1815,0.1804,0.1805,1719341.2000
1,2021-01-01 00:05:00,0.1805,0.1806,0.1791,0.1793,3551263.8000
2,2021-01-01 00:10:00,0.1793,0.1796,0.1783,0.1786,3737301.1000
3,2021-01-01 00:15:00,0.1787,0.1798,0.1785,0.1797,1539790.0000
4,2021-01-01 00:20:00,0.1797,0.1797,0.1789,0.1794,945063.5000
...,...,...,...,...,...,...
68309,2021-08-26 18:50:00,2.5400,2.5480,2.5360,2.5430,1127688.3000
68310,2021-08-26 18:55:00,2.5440,2.5450,2.5220,2.5270,930916.9000
68311,2021-08-26 19:00:00,2.5270,2.5330,2.5090,2.5110,1933868.6000
68312,2021-08-26 19:05:00,2.5100,2.5140,2.4890,2.4960,2967765.9400


In [9]:
data = data.set_index('timestamp')
data.index = pd.to_datetime(data.index)
data =  data.astype(float)
data

,open,high,low,close,volume
timestamp,,,,,
2021-01-01 00:00:00,0.1813,0.1815,0.1804,0.1805,1719341.2000
2021-01-01 00:05:00,0.1805,0.1806,0.1791,0.1793,3551263.8000
2021-01-01 00:10:00,0.1793,0.1796,0.1783,0.1786,3737301.1000
2021-01-01 00:15:00,0.1787,0.1798,0.1785,0.1797,1539790.0000
2021-01-01 00:20:00,0.1797,0.1797,0.1789,0.1794,945063.5000
...,...,...,...,...,...
2021-08-26 18:50:00,2.5400,2.5480,2.5360,2.5430,1127688.3000
2021-08-26 18:55:00,2.5440,2.5450,2.5220,2.5270,930916.9000
2021-08-26 19:00:00,2.5270,2.5330,2.5090,2.5110,1933868.6000


In [10]:
# data = data.iloc[65000:]
# data

In [11]:
features = ['account', 'open', 'high', 'low', 'close',
           'r', 'm', 'v', 'min', 'max', 'mami', 'mac', 'mic', 'atr', 'atr%',
           'avg_price', 'med_price', 'typ_price', 'wcl_price', 'upperband',
           'middleband', 'lowerband', 'sma_7', 'sma_25', 'sma_99', 'ema_7', 'ema_25', 'ema_99',
           'macd', 'macd_signal', 'macd_hist', 'rsi', 'cci', 'dx', 'units']

In [12]:
learn_env = CryptoEnv(initial_account=1e6, data=data, lag=lag, ptc=2.7e-4, mode='train', features=features)

In [13]:
test_env = CryptoEnv(initial_account=1e6, data=data, lag=lag, ptc=2.7e-4, mode='test', features=features)

In [14]:
# learn_env.draw_cumulative_return(agent.model)

In [15]:
# test_env.draw_cumulative_return(agent.model)

In [16]:
env_name = 'TradingEnv'

In [17]:
config = DEFAULT_CONFIG.copy()
config['num_gpus'] = n_gpus
config['tf_session_args']['device_count']['GPU'] = n_gpus
config['tf_session_args']['allow_soft_placement'] = True
config['tf_session_args']['gpu_options']['allow_growth'] = True

config['num_workers'] = 1
config['lr'] = 0.0003
config['gamma'] = 0.99
config['model']['fcnet_hiddens'] = [64, 64]
config['model']['fcnet_activation'] = 'tanh'
config['timesteps_per_iteration'] = len(learn_env.data)
config['train_batch_size'] = 8000
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed
config['framework']='tf'
config['env'] = env_name

config['double_q'] = False
config['dueling'] = False
config['num_atoms'] = 1
config['noisy'] = False
config['prioritized_replay'] = False
config['n_step'] = 1
config['target_network_update_freq'] = 8000
config['lr'] = .0000625
config['adam_epsilon'] = .00015
config['hiddens'] = [512]
config['learning_starts'] = 20000
config['buffer_size'] = 1000000
config['rollout_fragment_length'] = 4
config['train_batch_size'] = 32
config['exploration_config']['epsilon_timesteps'] = 200000
config['exploration_config']['final_epsilon'] = 0.01
config['prioritized_replay_alpha'] = 0.5
config['final_prioritized_replay_beta'] = 1.0
config['prioritized_replay_beta_annealing_timesteps'] = 2000000

In [18]:
def create_env(config):
    return learn_env

In [19]:
from ray.tune.registry import register_env

register_env(env_name, create_env)

In [20]:
max_training_episodes = 1000

In [21]:
# agent = DQNTrainer(config=config)

In [22]:
# while True:
#     result = agent.train()
#     if result['episodes_total'] % 100 == 0:
#         checkpoint = agent.save()
#         print("checkpoint saved at", checkpoint)
#     if result['episodes_total'] >= max_training_episodes:
#         break
# print('Mean Rewards:\t{:.1f}'.format(result['episode_reward_mean']))

In [23]:
# result

In [ ]:
# tune hyperparamters with grid search
# https://github.com/ray-project/ray/blob/master/python/ray/tune/tune.py
ray.init(ignore_reinit_error=True, num_cpus=n_cpus, num_gpus=n_gpus)
config = {
    "env": env_name,
    "num_workers": 10,
    'timesteps_per_iteration': len(learn_env.data),
#     'model': {
#         'fcnet_hiddens': tune.grid_search([[128, 128],[64, 64],[32, 32]])
#     },        
#     'train_batch_size': 1000,
#     "lr": tune.grid_search([0.0003, 0.0001]),
#     'gamma': tune.grid_search([0.99, 0.999,]),
    'framework': 'tf',
#     'num_gpus_per_worker': 1,
    'num_envs_per_worker': 50,
    
    'num_gpus': 1#n_gpus # using all gpus throw error, Have to fix this later  
}
                      
analysis = tune.run(
    "DQN",
    name='DQN',
    verbose=1,
    stop={"training_iteration": 10000},
    config = config,
    checkpoint_freq=10,
    checkpoint_at_end=True,
    checkpoint_score_attr='episode_reward_mean',
    local_dir=root_dir+'\\ray-results',
    progress_reporter=CLIReporter(),
    num_samples=1, 
#     resources_per_trial={"gpu": 1}
)


2021-08-27 04:54:07,453	INFO services.py:1247 -- View the Ray dashboard at http://127.0.0.1:8265
2021-08-27 04:54:17,282	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


== Status ==
Memory usage on this node: 13.7/63.9 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/24 CPUs, 0/3 GPUs, 0.0/31.06 GiB heap, 0.0/15.53 GiB objects
Result logdir: C:\Users\yosou\Documents\BooksWithCode\Binance\Deep Learning\Reinforcement-Learning\RL-ray\ray-results\DQN
Number of trials: 1/1 (1 PENDING)
+----------------------------+----------+-------+
| Trial name                 | status   | loc   |
|----------------------------+----------+-------|
| DQN_TradingEnv_6536b_00000 | PENDING  |       |
+----------------------------+----------+-------+




(pid=10800) C:\Users\yosou\Documents\BooksWithCode\Binance\Deep Learning\Reinforcement-Learning\RL-ray\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
(pid=10800)   'Module "zipline.assets" not found; multipliers will not be applied'
(pid=10800) 2021-08-27 04:54:21,804	INFO trainer.py:706 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=10800) 2021-08-27 04:54:21,804	INFO trainer.py:720 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=18172) C:\Users\yosou\Documents\BooksWithCode\Binance\Deep Learning\Reinforcement-Learning\RL-ray\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
(pid=18172)   'Module "zipline.assets" not found; multipliers will not be applied'
(pid=18160) C:\Users\yosou\Documents\BooksWithCode\Binance\Deep Learning\R

== Status ==
Memory usage on this node: 36.2/63.9 GiB
Using FIFO scheduling algorithm.
Resources requested: 11.0/24 CPUs, 1.0/3 GPUs, 0.0/31.06 GiB heap, 0.0/15.53 GiB objects
Result logdir: C:\Users\yosou\Documents\BooksWithCode\Binance\Deep Learning\Reinforcement-Learning\RL-ray\ray-results\DQN
Number of trials: 1/1 (1 RUNNING)
+----------------------------+----------+---------------------+--------+------------------+-------+----------+----------------------+----------------------+--------------------+
| Trial name                 | status   | loc                 |   iter |   total time (s) |    ts |   reward |   episode_reward_max |   episode_reward_min |   episode_len_mean |
|----------------------------+----------+---------------------+--------+------------------+-------+----------+----------------------+----------------------+--------------------|
| DQN_TradingEnv_6536b_00000 | RUNNING  | 192.168.2.103:10800 |      1 |          134.814 | 58000 |      nan |                  nan | 

== Status ==
Memory usage on this node: 30.5/63.9 GiB
Using FIFO scheduling algorithm.
Resources requested: 11.0/24 CPUs, 1.0/3 GPUs, 0.0/31.06 GiB heap, 0.0/15.53 GiB objects (0.0/1.0 CPU_group_4_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_1_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 GPU_group_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_7_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_0_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_2_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_9_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_10_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_3_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_5_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 GPU_group_0_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_6_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_8_029992d569bc05e3a218e4d87b5a8853, 0.0/11.0 CPU_group_029992d569bc05e3a218e4d87b5a8853)
Result logdir: C:\Users\yosou\Documents\BooksWithCode\Binance\Deep Le

...

== Status ==
Memory usage on this node: 27.9/63.9 GiB
Using FIFO scheduling algorithm.
Resources requested: 11.0/24 CPUs, 1.0/3 GPUs, 0.0/31.06 GiB heap, 0.0/15.53 GiB objects (0.0/1.0 CPU_group_4_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 GPU_group_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_1_029992d569bc05e3a218e4d87b5a8853, 0.0/11.0 CPU_group_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_7_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_10_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_3_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_6_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_5_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_8_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 GPU_group_0_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_9_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_2_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_0_029992d569bc05e3a218e4d87b5a8853)
Result logdir: C:\Users\yosou\Documents\BooksWithCode\Binance\Deep Le

== Status ==
Memory usage on this node: 28.1/63.9 GiB
Using FIFO scheduling algorithm.
Resources requested: 11.0/24 CPUs, 1.0/3 GPUs, 0.0/31.06 GiB heap, 0.0/15.53 GiB objects (0.0/1.0 CPU_group_4_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_1_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 GPU_group_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_7_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_0_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_2_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_9_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_10_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_3_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_5_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 GPU_group_0_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_6_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_8_029992d569bc05e3a218e4d87b5a8853, 0.0/11.0 CPU_group_029992d569bc05e3a218e4d87b5a8853)
Result logdir: C:\Users\yosou\Documents\BooksWithCode\Binance\Deep Le

== Status ==
Memory usage on this node: 28.3/63.9 GiB
Using FIFO scheduling algorithm.
Resources requested: 11.0/24 CPUs, 1.0/3 GPUs, 0.0/31.06 GiB heap, 0.0/15.53 GiB objects (0.0/1.0 CPU_group_0_029992d569bc05e3a218e4d87b5a8853, 0.0/11.0 CPU_group_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_2_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_1_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 GPU_group_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_7_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 GPU_group_0_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_6_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_8_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_9_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_10_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_3_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_4_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_5_029992d569bc05e3a218e4d87b5a8853)
Result logdir: C:\Users\yosou\Documents\BooksWithCode\Binance\Deep Le

== Status ==
Memory usage on this node: 28.2/63.9 GiB
Using FIFO scheduling algorithm.
Resources requested: 11.0/24 CPUs, 1.0/3 GPUs, 0.0/31.06 GiB heap, 0.0/15.53 GiB objects (0.0/1.0 CPU_group_4_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_1_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 GPU_group_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_7_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_0_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_2_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_9_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_10_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_3_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_5_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 GPU_group_0_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_6_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_8_029992d569bc05e3a218e4d87b5a8853, 0.0/11.0 CPU_group_029992d569bc05e3a218e4d87b5a8853)
Result logdir: C:\Users\yosou\Documents\BooksWithCode\Binance\Deep Le

== Status ==
Memory usage on this node: 28.2/63.9 GiB
Using FIFO scheduling algorithm.
Resources requested: 11.0/24 CPUs, 1.0/3 GPUs, 0.0/31.06 GiB heap, 0.0/15.53 GiB objects (0.0/1.0 CPU_group_4_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 GPU_group_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_1_029992d569bc05e3a218e4d87b5a8853, 0.0/11.0 CPU_group_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_7_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_10_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_3_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_6_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_5_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_8_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 GPU_group_0_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_9_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_2_029992d569bc05e3a218e4d87b5a8853, 0.0/1.0 CPU_group_0_029992d569bc05e3a218e4d87b5a8853)
Result logdir: C:\Users\yosou\Documents\BooksWithCode\Binance\Deep Le